In [ ]:
%run init.ipy
from utils import *
import miregularizer2

In [ ]:
from utils import load_mnist_rnn
d=load_mnist_rnn(max_train_items=2000, max_test_items=2000, normalize=False)
noiselayer, kdelayer = None, None

In [ ]:
from keras.models import Sequential
from keras.layers import RepeatVector, TimeDistributed # Dense, Dropout, Activation, , , 
from rnn import NoInputRNN

macro_dims = 15
num_output_vars, num_timesteps, num_input_vars = d.train.Y.shape[2], d.train.Y.shape[1], d.train.X.shape[1]
act1, act2, act3 = 'tanh', 'tanh', 'tanh'

model = Sequential()
model.add(Dense(macro_dims, activation=act1, input_dim=num_input_vars))
kdelayer, noiselayer = None, None
if True:
    kdelayer   = miregularizer2.KDEParamLayer(init_logvar=-5)
    noiselayer = miregularizer2.GaussianNoise2(d.train, init_logvar=-5, kdelayer=kdelayer, regularizemi=True, init_alpha=0.)
    #model.add(BatchNormalization())
    model.add(kdelayer)
    model.add(noiselayer)

model.add(RepeatVector(1))
# noise plus mi layer

model.add(NoInputRNN(output_dim=macro_dims, num_timesteps=num_timesteps))

#model.add(TDD(Dense(macro_dims, activation=act1), input_shape=(num_timesteps,num_input_vars)))
#model.add(rnn_w_noise(macro_dims, 
#                      U_regularizer=regobj, 
#                      activity_regularizer=regact, 
#                      return_sequences=True, 
#                      activation=act2,
#                      sigma=.1
#                     ))
model.add(TimeDistributed(Dense(num_output_vars, activation=act3)))

kdetraincb, noisetraincb, reportcb = None, None, None
if noiselayer is not None and kdelayer is not None:
    kdetraincb   = miregularizer2.KDETrain(traindata=d.train, kdelayer=kdelayer)
    noisetraincb = miregularizer2.NoiseTrain(traindata=d.train, noiselayer=noiselayer)
    reportcb     = miregularizer2.ReportVars(noiselayer=noiselayer, kdelayer=kdelayer)


#model.compile(loss='mse', optimizer='rmsprop')

In [ ]:
def compilefunc():
    model.compile(loss='mse', optimizer='rmsprop')
    if noiselayer is not None:
        K.set_value(noiselayer.logvar, -10)
    if kdelayer is not None:
        K.set_value(kdelayer.logvar, -10)
    
compilefunc()

In [ ]:
#%%timeit -n 1 -r 1
if True:
    batch_size = 10
    nbepoch = 100

    cbs = []
    if reportcb is not None:
        cbs.append(reportcb)
        
    if noiselayer is not None:
        K.set_value(noiselayer.alpha, 0.0)

    inithist = model.fit(d.train.X, d.train.Y, nb_epoch=nbepoch,
                     batch_size=batch_size, verbose=2, # validation_split=0.1, 
                     callbacks=cbs)

    model.save_weights("models/rnnfitmodel.h5")
    

In [ ]:
#alphavals = np.linspace(0, 5, 20, endpoint=True)
#alphavals = np.exp(np.linspace(0, np.log(6), 20, endpoint=True))-1
alphavals = np.exp(np.linspace(0, np.log(1.01), 10, endpoint=True))-1

nbepoch = 50
batch_size = 10
if 'saved_hist' not in locals():
    saved_hist = {}
for alpha in alphavals: # [0.0001, 0.0002, 0.01, 0.2, 0.3] :
    print "**************** Doing alpha=%.5f ****************" % alpha
    if alpha in saved_hist:
        continue
    # Reset
    #for clayer in model.layers:
    #    clayer.build(clayer.input_shape)
    
    compilefunc()
    model.load_weights("models/rnnfitmodel.h5")
    if noiselayer is not None :
        K.set_value(noiselayer.alpha, alpha)

    hist = model.fit(d.train.X, d.train.Y, nb_epoch=nbepoch,
                     batch_size=batch_size, verbose=2, # validation_split=0.1, 
                     callbacks=[kdetraincb, noisetraincb, reportcb,]
                    )
    saved_hist[alpha] = {'history':hist.history, 'endlogs': miregularizer2.get_logs(model, d, kdelayer, noiselayer)}
    fname = "models/rnnfitmodel%0.5f.h5"%alpha
    print "saving to %s"%fname
    model.save_weights(fname)

    print 
    print

In [ ]:
import cPickle
saved_hist2 = {}
for k,v in saved_hist.iteritems():
    if k > 0.01: 
        continue
    saved_hist2[k] = v
    
with open('rnnsavedhist.dat', 'wb') as f:
    cPickle.dump(saved_hist2, f)

In [ ]:
#with open('rnnsavedhist.dat', 'rb') as f:
#    saved_hist2 = cPickle.load(f)
    
matplotlib.rcParams.update({'font.size': 8, 'lines.linewidth':2})

alphavals, mivalstst, mivalstrn, klvalstst, klvalstrn = [], [], [], [], []

for alpha in sorted(saved_hist.keys()):
    if alpha >= 0.015: 
        continue
    hist = saved_hist[alpha]['endlogs']
    #ndx = -1 # np.argmin( hist['val_loss'] )
    alphavals.append(alpha)
    mivalstrn.append(hist['mi_trn'])
    mivalstst.append(hist['mi_tst'])
    klvalstst.append(hist['kl_tst'])
    klvalstrn.append(hist['kl_trn'])
    #valloss.append(hist['val_loss'][ndx])
    #print ndx
    
alphavals, mivalstst, mivalstrn, klvalstst, klvalstrn = map(np.array, [alphavals, mivalstst, mivalstrn, klvalstst, klvalstrn])
plt.figure(figsize=(14,4))
plt.subplot(1,4,1)
plt.scatter(alphavals, mivalstrn)
plt.xlabel('alpha')
plt.ylabel('MI-train')
plt.xlim([0, .01])
plt.ylim([0, plt.ylim()[1]])
plt.subplot(1,4,2)
plt.scatter(alphavals, mivalstst)
plt.xlabel('alpha')
plt.ylabel('MI-test')
plt.xlim([0, .01])
plt.ylim([0, plt.ylim()[1]])
plt.subplot(1,4,3)
plt.scatter(alphavals, klvalstrn)
plt.xlabel('alpha')
plt.ylabel('KL-train')
plt.xlim([0, .01])
plt.subplot(1,4,4)
plt.scatter(alphavals, klvalstst)
plt.xlabel('alpha')
plt.ylabel('KL-test')
plt.xlim([0, .01])

#plt.subplot(1,4,4)
#plt.scatter(alphavals, valloss)
#plt.xlabel('alpha')
#plt.ylabel('Valloss')

In [ ]:
print alphavals

In [ ]:
import matplotlib 
#with plt.style.context([u'seaborn-paper']):
if True:
    fig=plt.figure(figsize=(13,4))
    matplotlib.rcParams.update({'font.size': 16, 'lines.linewidth':2})
    ax1 = fig.add_subplot(121)
    ln2 = np.log(2)
    print ln2*mivalstrn
    plt.plot(ln2*mivalstrn,ln2*klvalstrn)
    plt.xlabel('MI (bits)')
    plt.ylabel('KL (bits)')
    plt.title('Training dataset')
    ax2 = fig.add_subplot(122, sharey=ax1)
    plt.plot(ln2*mivalstst, ln2*klvalstst)
    plt.xlabel('MI (bits)')
    plt.ylabel('KL (bits)')
    plt.title('Testing dataset')

plt.savefig('imgs/mnist.pdf',bbox_inches='tight')
#hist.history # ['val_loss']

In [ ]:
import matplotlib 
#with plt.style.context([u'seaborn-paper']):
if True:
    fig=plt.figure(figsize=(15,4))
    matplotlib.rcParams.update({'font.size': 16, 'lines.linewidth':1.5})
    plt.subplot(1,3,1)
    plt.plot(alphavals, mivalstrn, label='Training')
    plt.xlabel(r'$\alpha$')
    plt.ylabel('MI (bits)')
    plt.plot(alphavals, mivalstst, label='Testing')
    plt.ylim([0,8])
    plt.subplot(1,3,2)
    plt.plot(alphavals, klvalstrn, label='Training')
    plt.xlabel(r'$\alpha$')
    plt.ylabel('KL (bits)')
    plt.plot(alphavals, klvalstst, label='Testing')
    plt.subplot(1,3,3)
    
    ln2 = np.log(2)
    print ln2*mivalstrn
    plt.plot(ln2*mivalstrn,ln2*klvalstrn, label='Training')
    plt.xlabel('MI (bits)')
    plt.ylabel('KL (bits)')
    #plt.title('Training dataset')
    #ax2 = fig.add_subplot(122, sharey=ax1)
    plt.plot(ln2*mivalstst, ln2*klvalstst, label='Testing')
    #plt.xlabel('MI (bits)')
    #plt.ylabel('KL (bits)')
    #plt.title('Testing dataset')
    plt.legend()
    fig.tight_layout()
plt.savefig('imgs/mnist2.pdf',bbox_inches='tight')
#hist.history # ['val_loss']

In [ ]:
"""
#%%timeit -n 1 -r 1
batch_size = 10
callbacks = []
if noiselayer is not None and kdelayer is not None:
    noiselayer.alpha = 0.0
    callbacks = [miregularizer2.KDETrain(traindata=d, model=model, kdelayer=kdelayer),
                 miregularizer2.NoiseTrain(traindata=d, model=model, noiselayer=noiselayer),
                 miregularizer2.ReportVars(traindata=d, noiselayer=noiselayer,kdelayer=kdelayer),
                ]

hist = model.fit(d.X_train, d.Y_train, nb_epoch=10,
                 batch_size=batch_size, validation_split=0.1, verbose=2, 
                 callbacks=callbacks)
#print hist.history
"""
print

In [ ]:
for l, v in hist.history.iteritems():
    plt.figure()
    plt.plot(v)
    plt.title(l)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
#fitmodel1.00000.h5
#activity = {}
for fname in ["fitmodel1.50000.h5",]:#"fitmodel.h5", "fitmodel1.00000.h5", "fitmodel2.00000.h5"]:
    compilefunc()
    model.load_weights(fname)

    #activity = get_activations(model, -2, d.X_train)[0]
    #get_activations = K.function([model.layers[0].input, K.learning_phase()], [noiselayer.output,])
    #activations = get_activations([X_batch,0])
    #activity = get_activations([d.X_train, 1])
    #plt.figure()
    #plot_activity(activity, colors=d.y_train, size=1) # , doPCA=False)
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [noiselayer.input,])
    activity[fname]=get_activations([d.X_train,])[0]

In [ ]:
fig = plt.figure(figsize=(15,4))
for ndx, fname in enumerate(["fitmodel.h5", "fitmodel1.00000.h5","fitmodel2.00000.h5"]): #"fitmodel1.50000.h5", 
    ax = fig.add_subplot(131+ndx, projection='3d')
    #plt.subplot(1,4,ndx+1)
    plot_activity(activity[fname], doPCA=True, colors=d.y_train, size=2, dims=3, opts=dict(whiten=True))
    plt.title(r'$\alpha = %d$' %ndx)
    plt.xticks([])
    plt.yticks([])
#plt.savefig('imgs/hidden.pdf',bbox_inches='tight')

In [ ]:
print len(d2.X_test)


In [ ]:
plt.figure()
plt.scatter(activity[:,0],activity[:,1])
plt.xlim([-1.5,1.5])
plt.ylim([-1.5,1.5])
plt.figure()
plt.plot(np.var(activity, axis=0))
plt.ylim([0, plt.ylim()[1]])

"""
f1 = K.function([model.layers[0].input, K.learning_phase()], [noiselayer.input])
x1=f1([d.X_train, 0])[0]
from mpl_toolkits.mplot3d import Axes3D
fig=plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

import sklearn.decomposition

x2 = sklearn.decomposition.PCA(3, whiten=True).fit_transform(x1)
plt.scatter(x2[:,0], x2[:,1], zs=x2[:,2], c=d.y_train, edgecolor='none')
print x2.shape
"""

In [ ]:
#adict = {}
#adict['noreg'] = get_activations(model, 0, X_train)[0]
#adict['entropyonly'] = get_activations(model, 0, X_train)[0]
#adict['mionly'] = get_activations(model, 0, X_train)[0]
#adict['entropynnoise'] = get_activations(model, 0, X_train)[0]
#adict['minnoise'] = get_activations(model, 0, X_train)[0]
#adict['fixednoise'] = get_activations(model, 0, X_train)[0]
#adict['mifixednoise'] = get_activations(model, 0, X_train)[0]


In [ ]:
for k,v in adict.iteritems():
    plot_activity(v)
    plt.title(k)

In [ ]:
Kdists = K.placeholder(ndim=2)
Klogvar = K.placeholder(ndim=0)
from miregularizer2 import kde_entropy_from_dists_loo
N = 1000
dims = 10
lossfunc = K.function([Kdists, Klogvar,], [kde_entropy_from_dists_loo(Kdists, N, dims, K.exp(Klogvar))])


In [ ]:
lossfunc([np.eye(N),1])

In [ ]:
np.array(np.array([1])) # [0]

In [ ]:
fh=K.function([model.layers[0].input,],[noiselayer.mi_obj.get_h()])
f = K.function([model.layers[0].input,],[noiselayer.input])
#.mi_obj.get_h()])

In [ ]:
xx=f([d.X_train,])
print xx
print fh([d.X_train,])

In [ ]:
K.eval(K.exp(noiselayer.logvar)+K.exp(noiselayer.kdelayer.logvar))

In [ ]:
v=K.placeholder(ndim=2)
lv=K.placeholder(ndim=0)
from miregularizer2 import kde_entropy # , get_dims
f2 = K.function([v,lv],[kde_entropy(v,lv)]) # [noiselayer.mi_obj.get_h()])kde_entropy(self.layer.input, totalvar
            

In [ ]:
print xx[0].shape
f2([xx[0],1.])
#import theano
#theano.__version__

In [ ]:
alphavals = np.exp(np.linspace(0, np.log(1.1), 30, endpoint=True))-1
print alphavals


In [ ]:
def f1(output):
    y1 = K.expand_dims(output, 0)
    y2 = K.expand_dims(output, 1)

    dists = K.sum(K.square(y1-y2), axis=2) 
    return dists
#l2diff = tf.sqrt( tf.reduce_sum(tf.square(tf.sub(x1, x2)),
#                                reduction_indices=1))
in1 = K.placeholder()
f1k = K.function([in1], [f1(in1)])
import tensorflow as tf
def f2(x):
    #x1 = K.expand_dims(output, 0)
    #x2 = K.expand_dims(output, 1)

    #dists = K.sum(K.square(y1-y2), axis=2) 
    #dists = tf.reduce_sum(tf.square(tf.sub(x1, x2)),reduction_indices=2)
    
    x2 = K.expand_dims(K.sum(K.square(x), axis=1), 1)
    #x2 = x2 + K.transpose(x2)
    #return K.shape(x2)
    xy = K.dot(x, K.transpose(x))
    dists = x2 + K.transpose(x2) - 2*xy

    return dists
#in1 = K.placeholder()
f2k = K.function([in1], [f2(in1)])


In [ ]:
rrr=np.random.random((2000,50))
#print f1k([rrr,])
print f2k([rrr,])


In [ ]:
compilefunc()
model.load_weights("models/rnnfitmodel.h5")

In [ ]:
ndxs = [6,]# np.arange(1, dtype='int')
x,y = d.X_test, d.Y_test
#x,y = d.X_train, d.Y_train
for i,actual in zip(model.predict(x[ndxs]),y[ndxs]):
    plt.figure(figsize=(10,6))
    plt.subplot(1,2,1)
    plt.imshow(actual.reshape([20,28]), interpolation='nearest',cmap='Greys', aspect='auto')
    plt.xticks([])
    plt.title('Ground Truth')
    plt.subplot(1,2,2)
    plt.imshow(i.reshape([20,28]), interpolation='nearest', cmap='Greys', aspect='auto')
    plt.xticks([])
    plt.title('Predicted')
plt.savefig('imgs/imgtask.pdf', bbox_inches='tight')